# DIVERS Variance and Covariance Decomposition

## 1) Read in absolute abundance tables 
### (data_X and data_Y are technical replicates, data_Z is the second spatial replicate)


In [15]:
%Directory containing absolute abundance tables
file_dir = ['/Users/brianji/Documents/dv_lab/Manuscripts/noise/revisions/round2/reviewer_code/v3/absolute_abundances/'];

%data_X
T_X = readtable([file_dir 'data_X.txt'],'Delimiter','\t');
otu_ids = table2array(T_X(:,1));
tax = table2array(T_X(:,end));
data_X = table2array(T_X(:,2:end-1));
[Mx,Nx] = size(data_X);

%data_Y
T_Y = readtable([file_dir 'data_Y.txt'],'Delimiter','\t');
data_Y = table2array(T_Y(:,2:end-1));
[My,Ny] = size(data_Y);

%data_Z
T_Z = readtable([file_dir 'data_Z.txt'],'Delimiter','\t');
data_Z = table2array(T_Z(:,2:end-1));
[Mz,Nz] = size(data_Z);


## 2) Calculate marginal (total) means and variances of each OTU 

In [16]:
marg_means = [];
marg_vars = [];

% Perform 500 different re-sampling iterations

for i = 1:5e2
    
    data_X_perm = [];
    
    %Randomly draw a sample from data_X, data_Y or data_Z
    for j = 1:Nx
        flip = rand;
        if flip > (2/3)
            data_X_perm(:,j) = data_X(:,j);
        elseif flip > (1/3)
            data_X_perm(:,j) = data_Y(:,j);
        else
            data_X_perm(:,j) = data_Z(:,j);
        end
    end
    
    %Estimate mean and variance from this iteration
    marg_means(:,i) = mean(data_X_perm,2);
    marg_vars(:,i) = var(data_X_perm')';
end

%Average over all re-sampling iterations
means = mean(marg_means,2);
vars_total = mean(marg_vars,2);

## 3) Variance decomposition of OTU abundances (should take several seconds)


In [17]:
disp(['Performing variance decomposition...'])


covs_XZ = [];
covs_XmZY = [];
vars_XmY = [];

%Perform 500 re-sampling iterations
for i = 1:5e2

    data_X_perm = [];
    data_Y_perm = [];
    
    %Randomly permute X and Y, Z stays the same
    for j = 1:Nx
        flip = rand;
        if flip > .5
            data_X_perm(:,j) = data_X(:,j);
            data_Y_perm(:,j) = data_Y(:,j);
        else
            data_X_perm(:,j) = data_Y(:,j);
            data_Y_perm(:,j) = data_X(:,j);
        end
    end
    
    %Temporal variance
    cov_XZ = [];
    for j = 1:Mx
        covmat_xz = cov(data_X_perm(j,:),data_Z(j,:));
        cov_xz = covmat_xz(1,2);
        cov_XZ(j) = cov_xz;
    end
    covs_XZ(:,i) = cov_XZ';
    
    %Spatial sampling variance
    cov_XmZY = [];
    for j = 1:Mx
        covmat_xmzy = cov(data_X_perm(j,:)-data_Z(j,:),data_Y_perm(j,:));
        cov_xmzy = covmat_xmzy(1,2);
        cov_XmZY(j) = cov_xmzy;
    end
    covs_XmZY(:,i) = cov_XmZY';
    
    %Technical
    var_XmY = .5 * var((data_X_perm - data_Y_perm)')';
    vars_XmY(:,i) = var_XmY;
    
end

%% Average over re-sampling iterations

%Temporal variance
vars_T = mean(covs_XZ,2);
vars_T = max(vars_T,0); 

%Spatial sampling variance
vars_S = mean(covs_XmZY,2);
vars_S = max(vars_S,0); 

%Technical variance
vars_N = mean(vars_XmY,2);

%Fraction of each component to total variance
vf_N = vars_N ./ (vars_N + vars_S + vars_T);
vf_S = vars_S ./ (vars_N + vars_S + vars_T);
vf_T = vars_T ./ (vars_N + vars_S + vars_T);


disp(['Variance decomposition complete!'])

Performing variance decomposition...
Variance decomposition complete!


## 4) Covariance decomposition of OTU abundances


### a) Calculate total covariances between all OTU pairs

In [18]:
disp(['Calculating total covariances...'])

covs_total = [];

%Perform 500 re-sampling iterations

for i = 1:5e2

    data_X_perm = [];

    %Randomly choose a sample from of data_X, data_Y, data_Z
    for j = 1:Nx
        flip = rand;
        if flip > (2/3)
            data_X_perm(:,j) = data_X(:,j);
        elseif flip > (1/3)
            data_X_perm(:,j) = data_Y(:,j);
        else
            data_X_perm(:,j) = data_Z(:,j);
        end
    end
    
    %Mean center the data 
    data_X_mc = data_X_perm - repmat(mean(data_X_perm,2),1,Nx);
    
    %Calculate covariance matrix
    covmat_X_perm = 1/(Nx-1) * data_X_mc * data_X_mc';
    
    %Store covariance matrix for this iteration
    covs_total(:,:,i) = covmat_X_perm;  
    
end
covmat_total = mean(covs_total,3);
clear covs_total;

disp(['Total covariances calculated!'])


Calculating total covariances...
Total covariances calculated!


### b) Calculate temporal covariances between all OTU pairs (should take several seconds)

In [19]:
disp(['Calculating temporal covariances...'])

crosscovs_T = [];

%Perform 500 re-sampling iterations
for i = 1:5e2

    data_X_perm = [];
    data_Y_perm = [];
    data_Z_perm = data_Z;
    
    %One realization of the data
    for j = 1:Nx
        flip = rand;
        if flip > .5
            data_X_perm(:,j) = data_X(:,j);
            data_Y_perm(:,j) = data_Y(:,j);
        else
            data_X_perm(:,j) = data_Y(:,j);
            data_Y_perm(:,j) = data_X(:,j);
        end
    end
    
    %Mean center matrices;
    data_X_mc = data_X_perm - repmat(mean(data_X_perm,2),1,Nx);
    data_Y_mc = data_Y_perm - repmat(mean(data_Y_perm,2),1,Ny);
    data_Z_mc = data_Z_perm - repmat(mean(data_Z_perm,2),1,Nz);
    
    %Calculate all pairwise covariances for this realization (2
    %permutations)
    covmat_XZ = 1/(Nx-1) * data_X_mc*data_Z_mc'; %(X_i,Z_j)
    covmat_ZX = 1/(Nx-1) * data_Z_mc*data_X_mc'; %(Z_i,X_j)
    covmat = 1/2 * (covmat_XZ + covmat_ZX);
    
    %Store covmat for each realization
    crosscovs_T(:,:,i) = covmat;    
    
end

%Average over all iterations
covmat_T = mean(crosscovs_T,3);
clear crosscovs_T;

disp(['Temporal covariances calculated!'])


Calculating temporal covariances...
Temporal covariances calculated!


### c) Calculate spatial sampling covariances between all OTU pairs (should take several seconds)

In [20]:
disp(['Calculating spatial covariances...'])


crosscovs_S = [];

%Peform 500 re-sampling iterations
for i = 1:5e2

    data_X_perm = [];
    data_Y_perm = [];
    data_Z_perm = data_Z;
    
    %One realization of the data
    for j = 1:Nx
        flip = rand;
        if flip > .5
            data_X_perm(:,j) = data_X(:,j);
            data_Y_perm(:,j) = data_Y(:,j);
        else
            data_X_perm(:,j) = data_Y(:,j);
            data_Y_perm(:,j) = data_X(:,j);
        end
    end
    
    data_XmZ = data_X_perm - data_Z_perm;
    
    %Mean center matrices;
    data_XmZ_mc = data_XmZ - repmat(mean(data_XmZ,2),1,Nx);
    data_Y_mc = data_Y_perm - repmat(mean(data_Y_perm,2),1,Ny);
    
    %Calculate all pairwise covariances for this realization (2
    %permutations)
    covmat_XZY = 1/(Nx-1) * data_XmZ_mc*data_Y_mc';
    covmat_YXZ = 1/(Nx-1) * data_Y_mc*data_XmZ_mc';
    covmat = 1/2 * (covmat_XZY + covmat_YXZ);
    
    %Store covmat for each realization
    crosscovs_S(:,:,i) = covmat;  
    
end

%Average over all iterations
covmat_S = mean(crosscovs_S,3);
clear crosscovs_S;


disp(['Spatial covariances calculated!'])

Calculating spatial covariances...
Spatial covariances calculated!


### d) Calculate technical covariances between all OTU pairs (should take several seconds)

In [21]:
disp(['Calculating technical covariances...'])

crosscovs_N = [];

%Peform 500 re-sampling iterations
for i = 1:5e2

    data_X_perm = [];
    data_Y_perm = [];
    data_Z_perm = data_Z;
    
    %One realization of the data
    for j = 1:Nx
    flip = rand;
        if flip > .5
            data_X_perm(:,j) = data_X(:,j);
            data_Y_perm(:,j) = data_Y(:,j);
        else
            data_X_perm(:,j) = data_Y(:,j);
            data_Y_perm(:,j) = data_X(:,j);
        end
    end
   
    data_XmY = data_X_perm - data_Y_perm;
    
    %Mean center matrices;
    data_XmY_mc = data_XmY - repmat(mean(data_XmY,2),1,Nx);
    
    %Calculate all pairwise covariances for this realization (1
    %permutation)
    covmat = 1/(Nx-1) * data_XmY_mc*data_XmY_mc';
    
    %Store covmat for each realization
    crosscovs_N(:,:,i) = .5 * covmat;   
   
end

%Average over all iterations
covmat_N = mean(crosscovs_N,3);  
clear crosscovs_N;

disp(['Technical covariances calculated!'])


Calculating technical covariances...
Technical covariances calculated!


## 5) Rescale covariances to obtain correlations

In [22]:
L = length(vars_total);

%Calculate product of marginal standard deviations for each pair of OTUs
sigxsigy = [];
for i = 1:L
   for j = 1:i
       if vars_total(i) > 0 && vars_total(j) > 0
            sigxsigy(i,j) = sqrt(vars_total(i))*sqrt(vars_total(j));
            sigxsigy(j,i) = sqrt(vars_total(i))*sqrt(vars_total(j));
       else
            sigxsigy(i,j) = 0;
            sigxsigy(j,i) = 0;
       end
   end
end

cormat_total = covmat_total ./ sigxsigy;
cormat_T = covmat_T ./ sigxsigy;
cormat_S = covmat_S ./ sigxsigy;
cormat_N = covmat_N ./ sigxsigy;


    
clear covs_XZ covs_XmZY vars_XmY sigxsigy data_X_perm data_Y_perm data_Z_perm data_X_mc data_Y_mc data_Z_mc data_XmY_mc data_XmZ_mc data_XmY data_XmZ



## 6) Saving

In [12]:
%Save DIVERS variance and covariance decomposition results into a MAT file
save_dir = ['/Users/brianji/Documents/dv_lab/Manuscripts/noise/revisions/round2/reviewer_code/v3/matData/'];
save([save_dir 'DIVERS_gut.mat']);
